Importing the datasets and libraries.

In [1]:
import pandas as pd
import numpy as np

slidings = pd.read_csv('Data/slidings_merged.csv')
rain = pd.read_csv('Data/rain_prepared.csv')

Taking a look at the datasets.

In [2]:
slidings = slidings.drop(columns='Unnamed: 0',axis=1)
slidings.head()

,situation,Date,location,risk,latitude,longitude,victims,deadly_victims,length,num_points,in_place,sliding
0,1,2019-12-10,1,0,0.0,0.0,0,0,0,0,0,1
1,1,2019-12-10,1,0,0.0,0.0,0,0,0,0,0,1
2,1,2019-09-17,1,0,0.0,0.0,0,0,0,0,0,1
3,1,2019-09-09,1,0,0.0,0.0,0,0,0,0,0,1
4,1,2019-06-17,1,0,0.0,0.0,0,0,0,0,0,1


In [3]:
rain = rain.drop(columns='Unnamed: 0',axis=1)
rain.head()

,Station,rain,latitude,longitude,Date
0,Recife (Lamepe/Itep),0.0,-8.0592,-34.9519,2012-01-01
1,Recife (Santo Amaro),0.0,-8.0503,-34.8842,2012-01-01
2,Recife (Várzea),0.0,-8.0360,-34.9800,2012-01-01
3,Recife - PCD,0.0,-8.0644,-34.9247,2012-01-01
4,Recife (Alto da Brasileira),0.0,-8.0014,-34.9353,2012-02-01


In [4]:
rain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8017 entries, 0 to 8016
Data columns (total 5 columns):
Station      8017 non-null object
rain         8017 non-null float64
latitude     8017 non-null float64
longitude    8017 non-null float64
Date         8017 non-null object
dtypes: float64(3), object(2)
memory usage: 250.6+ KB


Creating the rain column to fill with the value from the nearest station on that day. Those stations with no latitude/longitude values, will be filled with the average of the stations on the respective day.

In [5]:
slidings['Rain']=0

In [6]:
from geopy.distance import geodesic
for n in range (0,len(slidings)):
    b= rain.loc[rain['Date']==slidings.Date.iloc[n]]
    p1 = [slidings['latitude'][n],slidings['longitude'][n]]
    point =[]
    distance = []
    if slidings.latitude[n] == 0:
        slidings.Rain.iloc[n]= b['rain'].sum()/len(b['rain'])
    else:
            for i in range (0,len(b)):
                point = [b.latitude.iloc[i],b.longitude.iloc[i]]
                distance.append(geodesic(p1, point).meters)
            slidings.Rain.iloc[n]=b.rain.iloc[distance.index(min(distance))]

            
slidings['Rain']= np.round(slidings['Rain'],decimals=2)

c:\users\ricardo_2\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
c:\users\ricardo_2\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


In [7]:
slidings.Rain.value_counts()

0.00      11590
0.60        657
0.20        591
134.15      501
1.00        445
          ...  
34.05         1
0.92          1
13.26         1
10.25         1
1.95          1
Name: Rain, Length: 830, dtype: int64

The date, latitude and longitude columns are no longer needed so we will drop them. We will use the location list as the feature containing the location information, since it is the only that it is complete.

In [8]:
slidings = slidings.drop(columns=['latitude','longitude','Date'],axis=1)
slidings.head()

,situation,location,risk,victims,deadly_victims,length,num_points,in_place,sliding,Rain
0,1,1,0,0,0,0,0,0,1,0.60
1,1,1,0,0,0,0,0,0,1,0.60
2,1,1,0,0,0,0,0,0,1,0.13
3,1,1,0,0,0,0,0,0,1,2.17
4,1,1,0,0,0,0,0,0,1,93.60


Quick check for any null values. Since it could happen that no station had recorded values for rain on the day that we looked up.

In [9]:
slidings.isnull().any()

situation         False
location          False
risk              False
victims           False
deadly_victims    False
length            False
num_points        False
in_place          False
sliding           False
Rain               True
dtype: bool

In [10]:
slidings[slidings.isnull().any(axis=1)]['Rain']

3824   NaN
Name: Rain, dtype: float64

Removing the null value.

In [11]:
slidings = slidings.drop([3824])
slidings.isnull().any()

situation         False
location          False
risk              False
victims           False
deadly_victims    False
length            False
num_points        False
in_place          False
sliding           False
Rain              False
dtype: bool

In [12]:
slidings.to_csv(path_or_buf='slidings_training.csv')